In [36]:
import boto3
import pandas as pd
from io import StringIO 

In [37]:
AWS_ACCESS_KEY = "##############"
AWS_SECRET_KEY = "###########################"
AWS_REGION = "eu-north-1"
SCHEMA_NAME = "covid-19"
S3_STAGING_DIR = "s3://ayomikun-test-bucket/output/"
S3_BUCKET_NAME = "ayomikun-test-bucket"
S3_OUTPUT_DIRECTORY = "output"

In [38]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)

In [39]:
Dict = {}
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            #This function loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
        temp_file_location: str = "athena_query_results.csv"
        s3_client = boto3.client(
            "s3",
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_KEY,
            region_name=AWS_REGION,
        )
        s3_client.download_file(
            S3_BUCKET_NAME,
            f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
            temp_file_location,
        )
        return pd.read_csv(temp_file_location)

In [40]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhu_csv",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

In [41]:
response

{'QueryExecutionId': '2ec9663d-5ea0-48b8-be55-0428c1f56213',
 'ResponseMetadata': {'RequestId': '25d396e9-4d9e-457b-adae-99b14e9da19c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Apr 2024 03:07:18 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '25d396e9-4d9e-457b-adae-99b14e9da19c'},
  'RetryAttempts': 0}}

In [58]:
enigma_jhu_csv = download_and_load_query_results(athena_client, response)

In [59]:
import awswrangler as wr

# Use the provided S3 output path for storing Athena query results
s3_output = "s3://ayomikun-test-bucket/output/"

# Execute the query and read results into a pandas DataFrame using the provided S3 output path
df = wr.athena.read_sql_query(
    sql="SELECT * FROM enigma_jhu_csv",
    database="covid-19",
    s3_output=s3_output  # Use the provided S3 output path
)

# Display the first few rows of the DataFrame
print(df.head())


   fips admin2 province_state country_region          last_update  latitude  \
0  <NA>                 Anhui          China  2020-01-22T17:00:00    31.826   
1  <NA>               Beijing          China  2020-01-22T17:00:00    40.182   
2  <NA>             Chongqing          China  2020-01-22T17:00:00    30.057   
3  <NA>                Fujian          China  2020-01-22T17:00:00    26.079   
4  <NA>                 Gansu          China  2020-01-22T17:00:00    36.061   

   longitude  confirmed  deaths  recovered active combined_key  
0    117.226          1    <NA>       <NA>              "Anhui  
1    116.414         14    <NA>       <NA>            "Beijing  
2    107.874          6    <NA>       <NA>          "Chongqing  
3    117.987          1    <NA>       <NA>             "Fujian  
4    103.834       <NA>    <NA>       <NA>              "Gansu  


In [44]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM nytimes_data_in_usa_us_county",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)


In [45]:
response

{'QueryExecutionId': 'a2677c31-1e05-4f7c-b484-e02a399b54a9',
 'ResponseMetadata': {'RequestId': '81815938-3746-4c23-8b3e-84198216fa20',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Apr 2024 03:07:48 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '81815938-3746-4c23-8b3e-84198216fa20'},
  'RetryAttempts': 0}}

In [50]:
nytimes_data_in_usa_us_county = download_and_load_query_results(athena_client, response)

In [53]:
!pip install awswrangler


     ------------------------------------ 380.1/380.1 kB 493.1 kB/s eta 0:00:00


In [57]:
import awswrangler as wr

# Use the provided S3 output path for storing Athena query results
s3_output = "s3://ayomikun-test-bucket/output/"

# Execute the query and read results into a pandas DataFrame using the provided S3 output path
df = wr.athena.read_sql_query(
    sql="SELECT * FROM nytimes_data_in_usa_us_county",
    database="covid-19",
    s3_output=s3_output  # Use the provided S3 output path
)

# Display the first few rows of the DataFrame
print(df.head())


         date     county       state   fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061      1       0
1  2020-01-22  Snohomish  Washington  53061      1       0
2  2020-01-23  Snohomish  Washington  53061      1       0
3  2020-01-24       Cook    Illinois  17031      1       0
4  2020-01-24  Snohomish  Washington  53061      1       0


In [61]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM nytimes_data_in_usa_us_states",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)


In [62]:
response

{'QueryExecutionId': '54216b09-ce97-4c42-a8b6-f89662a50dc1',
 'ResponseMetadata': {'RequestId': 'cbdc1820-9c21-4379-ae86-c84392849514',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Apr 2024 11:21:27 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cbdc1820-9c21-4379-ae86-c84392849514'},
  'RetryAttempts': 0}}

In [63]:
nytimes_data_in_usa_us_states = download_and_load_query_results(athena_client, response)

In [75]:
s3_output = "s3://ayomikun-test-bucket/output/"

# Execute the query and read results into a pandas DataFrame using the provided S3 output path
df = wr.athena.read_sql_query(
    sql="SELECT * FROM nytimes_data_in_usa_us_states",
    database="covid-19",
    s3_output=s3_output  # Use the provided S3 output path
)

# Display the first few rows of the DataFrame
print(df.head())

         date       state  fips  cases  deaths
0  2020-01-21  Washington    53      1       0
1  2020-01-22  Washington    53      1       0
2  2020-01-23  Washington    53      1       0
3  2020-01-24    Illinois    17      1       0
4  2020-01-24  Washington    53      1       0


In [77]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_states_dailystates_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)


In [78]:
rearc_covid_19_testing_data_states_dailystates_daily = download_and_load_query_results(athena_client, response)

In [79]:
s3_output = "s3://ayomikun-test-bucket/output/"

# Execute the query and read results into a pandas DataFrame using the provided S3 output path
df = wr.athena.read_sql_query(
    sql="SELECT * FROM rearc_covid_19_testing_data_states_dailystates_daily",
    database="covid-19",
    s3_output=s3_output  # Use the provided S3 output path
)

# Display the first few rows of the DataFrame
print(df.head())

       date state  positive  probablecases  negative  pending  \
0  20210307    AK     56886           <NA>      <NA>     <NA>   
1  20210307    AL    499819         107742   1931711     <NA>   
2  20210307    AR    324818          69092   2480716     <NA>   
3  20210307    AS         0           <NA>      2140     <NA>   
4  20210307    AZ    826454          56519   3073010     <NA>   

  totaltestresultssource  totaltestresults  hospitalizedcurrently  \
0        totalTestsViral           1731628                     33   
1  totalTestsPeopleViral           2323788                    494   
2        totalTestsViral           2736442                    335   
3        totalTestsViral              2140                   <NA>   
4        totalTestsViral           7908105                    963   

   hospitalizedcumulative  ...  dataqualitygrade  deathincrease  \
0                    1293  ...                                0   
1                   45976  ...                              

In [80]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_dataus_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)


In [81]:
response

{'QueryExecutionId': 'a188ce74-43c0-4c26-b926-d08abaf47bb7',
 'ResponseMetadata': {'RequestId': '30b6fac1-ffc6-4149-9372-d81cab497e1f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Apr 2024 11:42:40 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '30b6fac1-ffc6-4149-9372-d81cab497e1f'},
  'RetryAttempts': 0}}

In [82]:
rearc_covid_19_testing_dataus_daily = download_and_load_query_results(athena_client, response)

In [83]:
s3_output = "s3://ayomikun-test-bucket/output/"

# Execute the query and read results into a pandas DataFrame using the provided S3 output path
df = wr.athena.read_sql_query(
    sql="SELECT * FROM rearc_covid_19_testing_dataus_daily",
    database="covid-19",
    s3_output=s3_output  # Use the provided S3 output path
)

# Display the first few rows of the DataFrame
print(df.head())

       date  states  positive  negative  pending  hospitalizedcurrently  \
0  20210307      56  28755524  74579770    11808                  40212   
1  20210306      56  28714259  74449356    11783                  41401   
2  20210305      56  28654639  74307155    12213                  42541   
3  20210304      56  28585852  74035238    12405                  44172   
4  20210303      56  28520365  73857281    11778                  45462   

   hospitalizedcumulative  inicucurrently  inicucumulative  \
0                  878613            8137            45475   
1                  877887            8409            45453   
2                  877384            8634            45373   
3                  874603            8970            45293   
4                  873073            9359            45214   

   onventilatorcurrently  ...          lastmodified recovered  total  posneg  \
0                   2801  ...  2021-03-07T24:00:00Z                0       0   
1               

In [85]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_dataus_total_latest",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)


In [86]:
response

{'QueryExecutionId': '7f179a4f-cf46-45f5-8fad-ff11a2d42fa9',
 'ResponseMetadata': {'RequestId': 'f2c01665-ff05-406c-93a9-246d5073b304',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Apr 2024 11:47:27 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f2c01665-ff05-406c-93a9-246d5073b304'},
  'RetryAttempts': 0}}

In [87]:
rearc_covid_19_testing_dataus_total_latest = download_and_load_query_results(athena_client, response)

In [88]:
s3_output = "s3://ayomikun-test-bucket/output/"

# Execute the query and read results into a pandas DataFrame using the provided S3 output path
df = wr.athena.read_sql_query(
    sql="SELECT * FROM rearc_covid_19_testing_dataus_total_latest",
    database="covid-19",
    s3_output=s3_output  # Use the provided S3 output path
)

# Display the first few rows of the DataFrame
print(df.head())

   positive  negative  pending  hospitalizedcurrently  hospitalizedcumulative  \
0   1061101   5170081     2775                  53793                  111955   

   inicucurrently  inicucumulative  onventilatorcurrently  \
0            9486             4192                   4712   

   onventilatorcumulative  recovered  \
0                     373     153947   

                                       hash              lastmodified  death  \
0  95064ba29ccbc20dbec397033dfe4b1f45137c99  2020-05-01T09:12:31.891Z  57266   

   hospitalized    total  totaltestresults   posneg             notes  
0        111955  6233957           6231182  6231182  "NOTE: ""total""  


In [101]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datastate_abv",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

In [102]:
response

{'QueryExecutionId': '5cc67b9c-03d2-453b-89a5-8d57ef80d589',
 'ResponseMetadata': {'RequestId': '60dad129-51df-4b5d-9ceb-80860ced33ca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Apr 2024 12:00:16 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '60dad129-51df-4b5d-9ceb-80860ced33ca'},
  'RetryAttempts': 0}}

In [103]:
static_datastate_abv = download_and_load_query_results(athena_client, response)

In [104]:
s3_output = "s3://ayomikun-test-bucket/output/"

# Execute the query and read results into a pandas DataFrame using the provided S3 output path
df = wr.athena.read_sql_query(
    sql="SELECT * FROM static_datastate_abv",
    database="covid-19",
    s3_output=s3_output  # Use the provided S3 output path
)

# Display the first few rows of the DataFrame
print(df.head())

       col0          col1
0     State  Abbreviation
1   Alabama            AL
2    Alaska            AK
3   Arizona            AZ
4  Arkansas            AR
